In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = \
  '--conf spark.cassandra.connection.host=cassandra --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,com.datastax.spark:spark-cassandra-connector_2.11:2.0.2 pyspark-shell'

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
# from pyspark.streaming.kafka import KafkaUtils
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [3]:
import json

In [4]:
import findspark
findspark.init()

In [5]:
# from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [6]:
df_raw = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "BDA_Project") \
  .option("includeHeaders", "true") \
  .load()
# df_raw.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
df_json = df_raw.selectExpr('CAST(value AS STRING) as json')
df_json.writeStream.format("console").start()

In [36]:
newJson = df_json
df = spark.read.json(newJson)
df.show(truncate=False)

TypeError: path can be only string, list or RDD

In [32]:
df_json['value']

AnalysisException: Cannot resolve column name "value" among (json);

In [30]:
words = df_json.select(
   F.explode(
       F.split(df_raw.value, " ")
   ).alias("word")
)

AnalysisException: Resolved attribute(s) value#738 missing from json#753 in operator !Generate explode(split(cast(value#738 as string),  , -1)), false, [word#1293].;;
Project [word#1293]
+- !Generate explode(split(cast(value#738 as string),  , -1)), false, [word#1293]
   +- Project [cast(value#738 as string) AS json#753]
      +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@7dc1d05e, kafka, org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaTable@73062196, org.apache.spark.sql.util.CaseInsensitiveStringMap@7aa3ad2e, [key#737, value#738, topic#739, partition#740, offset#741L, timestamp#742, timestampType#743, headers#744], StreamingRelation DataSource(org.apache.spark.sql.SparkSession@1c331790,kafka,List(),None,List(),None,Map(subscribe -> BDA_Project, includeHeaders -> true, kafka.bootstrap.servers -> kafka:9092),None), kafka, [key#729, value#730, topic#731, partition#732, offset#733L, timestamp#734, timestampType#735, headers#736]


In [28]:
wordCounts = words.groupBy("word").count()

In [29]:
wordCounts

DataFrame[word: string, count: bigint]

In [11]:
words

DataFrame[word: string]

In [19]:
schema_input = StructType([StructField('data1', StringType())])
schema_output = StructType([StructField('neg', StringType()),\
                            StructField('pos', StringType()),\
                            StructField('neu', StringType()),\
                            StructField('compound', StringType())])

df_raw.select(F.from_json(df_raw.json, schema_input).alias('sentence'))\
  .select('sentence.data1') \
  .writeStream \
  .format("console") \
  .start() 

AttributeError: 'DataFrame' object has no attribute 'json'

In [68]:
# display(df)
words = df.select(
   explode(
       split(df.value, " ")
   ).alias("word")
)

NameError: name 'explode' is not defined

In [23]:
print(df_json)

DataFrame[json: string]


In [17]:
# sc = SparkContext(appName="BigDataRiver")
# sc.setLogLevel("WARN")
# sc.setCheckpointDir('checkpoint/')
# ssc = StreamingContext(sc, 10)
# sql = SQLContext(sc)

In [ ]:
kafkaStream = KafkaUtils.createDirectStream(ssc, ['BDA_Project'], {"metadata.broker.list": 'kafka:9092'})

In [24]:
parsed = df_json.map(lambda v: v[1])

AttributeError: 'DataFrame' object has no attribute 'map'

In [ ]:
def usersWhoBoughtXAlsoBought(df):
    productDf = df.select('user_id', 'product')
    otherProductDf = productDf.toDF('user_id', 'other_product')
    matchedProductsDf = productDf.join(otherProductDf, otherProductDf['user_id'] == productDf['user_id'], 'inner').\
            filter("`product` != `other_product`").select('product','other_product').\
            groupby('product','other_product').count().toDF("product","other_product","count")
    return matchedProductsDf

In [ ]:
def selectTopProducts(df):
    df.registerTempTable("products")
    topProductsDf = sql.sql("""
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY product ORDER BY count DESC) rn
        FROM products
    """).where("rn <= 5").groupBy("product").agg(F.collect_list("other_product").alias("other_products"))
    return topProductsDf

In [ ]:
def processStream(rdd):
    df = sql.read.json(rdd)
    if(len(df.columns)):
        productDf = df.select('user_id', 'product').cache()
        selectTopProducts(usersWhoBoughtXAlsoBought(productDf)).\
            write.format("org.apache.spark.sql.cassandra").\
            mode('append').options(table="top_other_products_stream", keyspace="bdr").save()
            
        #store all user products for the batch process    
        productDf.write.format("org.apache.spark.sql.cassandra").\
            mode('append').options(table="all_user_products", keyspace="bdr").save()
            
        print "Done"
    else:
        print "Empty"

In [ ]:
parsed.foreachRDD(lambda rdd: processStream(rdd))

In [ ]:
ssc.start()
ssc.awaitTermination()